In [2]:
import numpy as np
import imageio as imo
import scipy.ndimage as img


def planerToPolar(x,y):
    r = np.sqrt(x**2+y**2)
    fi = np.arctan2(y,x)
    return r,fi

def polarToPlaner(r,fi):
    x = r*np.cos(fi)
    y = r*np.sin(fi)
    return x,y

def toRFiPlane(f,rows,cols,rmax,fimax):
    rs, fis = np.meshgrid(np.linspace(0,rmax,cols),
                         np.linspace(0,fimax,rows),
                         sparse = True)
    xs,ys = polarToPlaner(rs,fis)
    xs,ys = xs.reshape(-1), ys.reshape(-1)
    coords = np.vstack((ys,xs))
    g = img.map_coordinates(f, coords, order = 3)
    g = g.reshape(m, n)
    return np.flipud(g)
    
def toXYPlaneNaive(f, rows, cols):
    xs, ys = np.meshgrid(np.linspace(0,cols,cols),
                        np.linspace(0,rows,rows),
                        sparse = True)
    rs,fis = planerToPolar(xs,ys)
    rs,fis = rs.reshape(-1), fis.reshape(-1)
    coords = np.vstack((fis,rs))
    g = img.map_coordinates(f,coords, order = 3)
    g = g.reshape(m,n)
    return np.flipud(g)

def toXYPlaneCorrect(f, rows, cols, rmax, fimax):
    xs, ys = np.meshgrid(np.linspace(0,cols,cols),
                        np.linspace(0,rows,rows),
                        sparse = True)
    rs, fis = planerToPolar(xs,ys)
    rs, fis = rs.reshape(-1), fis.reshape(-1)
    #mapping rs and fis correctly
    iis = (fis/fimax)*(rows-1)
    jjs = (rs/rmax)*(cols-1)
    coords = np.vstack((iis,jjs))
    g = img.map_coordinates(f,coords, order = 3)
    g = g.reshape(rows,cols)
    return np.flipud(g)

# f = imo.imread('bauckhage.jpg')
# m,n = f.shape
# rmax = np.math.sqrt(m**2+n**2)
# fimax = np.pi/2;

# #flip the image first
# fimg = np.flipud(f)
# #convert to r,fi coords
# polarImg = toRFiPlane(fimg,m,n,rmax,fimax)
# #Save the img
# imo.imwrite('polarprof.jpg',polarImg)

# fimg = np.flipud(polarImg)

# normalNaive = toXYPlaneNaive(fimg,m,n)
# imo.imwrite('normalProf.jpg',normalNaive)

# normalCorrect = toXYPlaneCorrect(fimg,m,n,rmax,fimax)
# imo.imwrite('normalCorrect.jpg',normalCorrect)


In [5]:
def toRFiPlane2(f,rows,cols,rmax,fimax):
    rs, fis = np.meshgrid(np.linspace(0,rmax,cols),
                         np.linspace(0,fimax,rows),
                         sparse = True)
    xs,ys = polarToPlaner(rs,fis)
    xs,ys = xs+cols/2, ys+rows/2
    xs,ys = xs.reshape(-1),ys.reshape(-1)
    coords = np.vstack((ys,xs))
    
    g = img.map_coordinates(f,coords,order=3)
    g = g.reshape(rows,cols)
    return np.flipud(g)

def toXYPlane2(f,rows,cols,rmax,fimax):
    xs,ys = np.meshgrid(np.linspace(0,cols,cols),
                       np.linspace(0,rows,rows),
                       sparse = True)
    rs,fis = planerToPolar(xs-cols/2,ys-rows/2)
    rs,fis = rs.reshape(-1),fis.reshape(-1)
    
    #since arctan2 returns values in [-pi,pi] we need to add 2pi offset to negative values
    for i in range(0,(rows-1)*(cols-1)):
        if(fis[i]<0.0):
            fis[i] = fis[i] + 2*np.pi
    #mapping to correct rs and fis    
    iis = (fis/fimax)*(rows-1)
    jjs = (rs/rmax)*(cols-1)
    coords = np.vstack((iis,jjs))
    g = img.map_coordinates(f,coords,order=3)
    g = g.reshape(rows,cols)
    return np.flipud(g)

def ret1DGaussian(sigma):
    msize = int(np.ceil(sigma * 2.575) * 2 + 1)
    x = np.arange(msize)
    halfSize = (int)(msize/2)
    g = np.exp(-0.5 * ((x-halfSize) / sigma)**2)
    g /= g.sum()
    return g

def applyRowBlur(f,g):
    #convolve in one Direction
    tmp = np.ones_like(f)
    img.convolve1d(f,g,1,tmp,'wrap')
    return tmp

def applyColBlur(f,sigma):
    #convolve in one Direction
    tmp = np.ones_like(f)
    img.convolve1d(f,g,0,tmp,'wrap')
    return tmp
    
#Changing some parameter for r, fi plane
f = imo.imread('cat.png')
m,n = f.shape

rmax = np.sqrt((m/2)**2+(n/2)**2)
fimax = 2*np.pi

fimg = np.flipud(f)

polarImage2 = toRFiPlane2(fimg,m,n,rmax,fimax)
imo.imwrite('polarImage2.png',polarImage2)


#defining gaussian filter
sigma = 11
g = ret1DGaussian(sigma)

#apply 1D gaussian filter along Rs
polarImgRBlur = applyRowBlur(polarImage2,g)
imo.imwrite('polarImgRBlur'+str(sigma)+'.png',polarImgRBlur)
fimg = np.flipud(polarImgRBlur)
normalImage1 = toXYPlane2(fimg,m,n,rmax,fimax)
imo.imwrite('normalImage1'+str(sigma)+'.png',normalImage1)

#apply 1D gaussian filter along fis
polarImgFiBlur = applyColBlur(polarImage2,g)
imo.imwrite('polarImgFiBlur'+str(sigma)+'.png',polarImgFiBlur)
fimg = np.flipud(polarImgFiBlur)
normalImage2 = toXYPlane2(fimg,m,n,rmax,fimax)
imo.imwrite('normalImage2'+str(sigma)+'.png',normalImage2)

#applying Gaussian in both directions
#using gaussian blurred in R dirs for fi
polarImgBothBlur = applyColBlur(polarImgRBlur,g)
imo.imwrite('polarImgBothBlur'+str(sigma)+'.png',polarImgBothBlur)
fimg = np.flipud(polarImgBothBlur)
normalImageBoth2 = toXYPlane2(fimg,m,n,rmax,fimax)
imo.imwrite('normalImageBoth2'+str(sigma)+'.png',normalImageBoth2)


[[  0.           0.74780339   1.49560678 ... 171.24697627 171.99477966
  172.74258305]
 [  0.           0.7475764    1.49515279 ... 171.19499459 171.94257099
  172.69014738]
 [  0.           0.74689555   1.4937911  ... 171.03908111 171.78597666
  172.53287221]
 ...
 [  0.           0.74689555   1.4937911  ... 171.03908111 171.78597666
  172.53287221]
 [  0.           0.7475764    1.49515279 ... 171.19499459 171.94257099
  172.69014738]
 [  0.           0.74780339   1.49560678 ... 171.24697627 171.99477966
  172.74258305]]
